# KFP Challenge Lab 2

In this challenge lab, we'll extend the pipeline developed in the challenge lab 1 to add batch prediction.

**Learning Objectives:**
1. Learn how to handle `google-cloud-pipeline-components`.
1. Learn how to use Vertex AI batch prediction from KFP pipeline.

## Setup

In [ ]:
from datetime import datetime

from google.cloud import aiplatform

In [ ]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

In [ ]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

### Build the trainer image

In [ ]:
ARTIFACT_REGISTRY_DIR = "asl-artifact-repo"
IMAGE_NAME = "trainer_image_covertype_vertex"
IMAGE_TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"us-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REGISTRY_DIR}/{IMAGE_NAME}:{IMAGE_TAG}"
TRAINING_CONTAINER_IMAGE_URI

The notebook assumes the training container is already created under `us-docker.pkg.dev/takumiohym-sandbox/asl-artifact-repo/trainer_image_covertype_vertex`. You can find the image via the URL below if it exists.

In [ ]:
print(
    f"URL: https://console.cloud.google.com/artifacts/docker/{PROJECT_ID}/us/{ARTIFACT_REGISTRY_DIR}/{IMAGE_NAME}"
)

If the image doesn't exists, remove the comment out below and build it.

In [ ]:
# !gcloud builds submit --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI trainer_image_vertex

To match the ml framework version we use at training time while serving the model, we will have to supply the following serving container to the pipeline:

In [ ]:
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"
)

**Note:** If you change the version of the training ml framework you'll have to supply a serving container with matching version (see [pre-built containers for prediction](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)).

## Challenge Lab 2

In this lab, we'll extend the pipeline developed in [challenge_lab_1.ipynb](./challenge_lab_1.ipynb) to add additional component for bach prediction.<br>

To add this capability, let's modify the [`pipeline.py`](./pipeline_vertex/pipeline.py) in the following way.

Open [pipeline_vertex/pipeline.py](./pipeline_vertex/pipeline.py) and update these elements:
- **Import Necessary Objects**: Add import statements for necessary modules.
- **Add batch prediction component**: Add a batch prediction component to the pipeline definition.
  - Use BigQuery for both `instances_format` and `predictions_format`.
  - Use `f"bq://{PROJECT_ID}.covertype_dataset.validation"` for the `bigquery_source_input_uri` argument.
  - Use `f"bq://{PROJECT_ID}.covertype_dataset.batch_prediction_result"` for the `bigquery_destination_output_uri` argument.
  - Specify `"Cover_Type"` in the `excluded_fields` argument to remove labels.
  - Specify the other necessary arguments following [the API documents](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.8.0/api/v1/batch_predict_job.html?h=modelbatchpredictop#).

**Tips: Search `TODO 2` to locate the sections you need to update.**

### Reference:
- Prebuilt batch prediction component: https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.8.0/api/v1/batch_predict_job.html?h=modelbatchpredictop#
- Vertex AI batch prediction: https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#all-other-containers_4

### Expected Result
KFP Pipeline DAG (Extend the highlighted section):

<img src="https://github.com/GoogleCloudPlatform/asl-ml-immersion/assets/6895245/5e5eb3a8-ca13-4dda-92e2-43fe8e8dba94" width="500"/>


## Compile and run the pipeline

Let stat by defining the environment variables that will be passed to the pipeline compiler:

In [ ]:
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/data"

TRAINING_FILE_PATH = f"{DATA_ROOT}/training/dataset.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/dataset.csv"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BASE_OUTPUT_DIR = f"{ARTIFACT_STORE}/models/{TIMESTAMP}"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}
%env BASE_OUTPUT_DIR={BASE_OUTPUT_DIR}

Let us make sure that the `ARTIFACT_STORE` has been created, and let us create it if not:

In [ ]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

**Note:** In case the artifact store was not created and properly set before hand, you may need
to run in **CloudShell** the following command to allow Vertex AI to access it:

```
PROJECT_ID=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)")
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com" \
    --role="roles/storage.objectAdmin"
```

#### Use the CLI compiler to compile the pipeline

We compile the pipeline from the Python file we generated into a JSON description using the following command:

In [ ]:
PIPELINE_YAML = "covertype_kfp_pipeline-lab2.yaml"

In [ ]:
!kfp dsl compile --py pipeline_vertex/pipeline.py --output $PIPELINE_YAML

**Note:** You can also use the Python SDK to compile the pipeline:

```python
from kfp import compiler

compiler.Compiler().compile(
    pipeline_func=create_pipeline, 
    package_path=PIPELINE_YAML,
)

```

The result is the pipeline file. 

In [ ]:
!head {PIPELINE_YAML}

### Deploy and run the pipeline package

In [ ]:
EXPERIMENT_NAME = "kfp-covertype-experiment"

In [ ]:
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    experiment=EXPERIMENT_NAME,
    experiment_tensorboard=False,
)

pipeline = aiplatform.PipelineJob(
    display_name="covertype_kfp_pipeline_challenge_lab",
    template_path=PIPELINE_YAML,
    enable_caching=True,
)

pipeline.submit(experiment=EXPERIMENT_NAME)

### Check the Batch Prediction result
Let's take a look at the result. `prediction` columns should be populated as a prediction result.

In [ ]:
%%bigquery

select * from covertype_dataset.batch_prediction_result ORDER BY rand() LIMIT 10

Copyright 2024 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.